In [1]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np

pn.extension(design='material')

csv_file = ("https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv")
data = pd.read_csv(csv_file, parse_dates=["date"], index_col="date")

data.tail()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
date,,,,,,
2015-02-10 09:29:00,21.05,36.0975,433.0,787.250000,0.005579,1
2015-02-10 09:29:59,21.05,35.9950,433.0,789.500000,0.005563,1
2015-02-10 09:30:59,21.10,36.0950,433.0,798.500000,0.005596,1
2015-02-10 09:32:00,21.10,36.2600,433.0,820.333333,0.005621,1
2015-02-10 09:33:00,21.10,36.2000,447.0,821.000000,0.005612,1


In [2]:
def view_hvplot(avg, highlight):
    return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
        color="orange", padding=0.1, legend=False
    )

def find_outliers(variable="Temperature", window=30, sigma=10, view_fn=view_hvplot):
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = np.abs(residual) > std * sigma
    return view_fn(avg, avg[outliers])


In [3]:
find_outliers(variable='Temperature', window=20, sigma=10)

:Overlay
   .Curve.Temperature   :Curve   [date]   (Temperature)
   .Scatter.Temperature :Scatter   [date]   (Temperature)

In [4]:
variable_widget = pn.widgets.Select(name="variable", value="Temperature", options=list(data.columns))
window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)

In [5]:
bound_plot = pn.bind(find_outliers, variable=variable_widget, window=window_widget, sigma=sigma_widget)

In [6]:
first_app = pn.Column(variable_widget, window_widget, sigma_widget, bound_plot)

first_app

Column(design=<class 'panel.theme.materi...)
    [0] Select(design=<class 'panel.theme.materi..., name='variable', options=['Temperature', ...], value='Temperature')
    [1] IntSlider(design=<class 'panel.theme.materi..., end=60, name='window', start=1, value=30)
    [2] IntSlider(design=<class 'panel.theme.materi..., end=20, name='sigma', value=10)
    [3] ParamFunction(function, _pane=HoloViews, design=<class 'panel.theme.materi...)